In [1]:
import numpy as np
from scipy.stats import binned_statistic

In [2]:
names = ["time", "night", "filter"]
types = [float, int, "<U1"]
data = np.zeros(5, dtype=list(zip(names,types)))

data["filter"] = ["g", "r", "r", "g", "r"]
data["night"] = [0,0,0,1,1]
data["time"] = [0,.04,.5, 1.2, 1.3]

In [3]:
data

array([(0.  , 0, 'g'), (0.04, 0, 'r'), (0.5 , 0, 'r'), (1.2 , 1, 'g'),
       (1.3 , 1, 'r')],
      dtype=[('time', '<f8'), ('night', '<i8'), ('filter', '<U1')])

In [4]:
def color_and_slope(in_data, color_length=1./24, slope_length=3./24.):
    has_color = False
    has_slope = False
    u_filters = np.unique(in_data["filter"])
    
    for filtername in u_filters:
        in_filt = np.where(in_data["filter"] == filtername)[0]
        time_gaps = in_data["time"][in_filt] - in_data["time"][in_filt][np.newaxis].T
        if np.max(time_gaps) >= slope_length:
            has_slope = True
            break

    for filtername1 in u_filters:
        for filtername2 in u_filters:
            if filtername1 != filtername2:
                in_filt1 = np.where(in_data["filter"] == filtername1)[0]
                in_filt2 = np.where(in_data["filter"] == filtername2)[0]
                time_gaps = (
                    in_data["time"][in_filt1] - in_data["time"][in_filt2][np.newaxis].T
                )
                time_gaps = time_gaps[np.where(time_gaps > 0)]
                if time_gaps.size > 0:
                    if np.min(time_gaps[np.where(time_gaps > 0)]) <= color_length:
                        has_color = True
                        break
    if has_color & has_slope:
        return 1
    else:
        return 0

In [5]:
good = np.where(data["night"] == 0)
color_and_slope(data[good])

1

In [6]:
bins = [-.5, 0.5, 1.5]
stat, be, bn = binned_statistic(data["night"], data, statistic=color_and_slope, bins=bins)
stat

array([1., 0.])